In [1]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

In [4]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.9 MB/s eta 0:00:00


In [4]:
import pandas as pd

In [1]:
import sentencepiece

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW

# Example dataset
class InterviewQuestionsDataset(Dataset):
    def __init__(self, tokenizer, data, max_length=512):
        self.tokenizer = tokenizer
        self.input_texts = [f"topic: {item['topic']} </s>" for item in data]
        self.target_texts = [item['interview question'] for item in data]
        self.max_length = max_length

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        source = self.tokenizer.encode_plus(self.input_texts[idx], max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        target = self.tokenizer.encode_plus(self.target_texts[idx], max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        return source['input_ids'].squeeze(), target['input_ids'].squeeze()

# Prepare your data
data = [{'topic': 'random forest', 'interview question': 'What is a random forest?'}]
tokenizer = T5Tokenizer.from_pretrained('t5-small')
dataset = InterviewQuestionsDataset(tokenizer, data)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Load T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-small')
model.train()

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids, labels = batch
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}: Loss {loss.item()}")

# Save the model
model.save_pretrained('./my_t5_model')


In [9]:
pd.read_csv('augmented_questions.csv')

,topic,Questions
0,Random Forest,Do you know what a Random Forest is and how it...
1,Random Forest,Can you tell me about the principles behind Ra...
2,Random Forest,Do you know what a Random Forest is and how it...
3,Random Forest,Do you know what a Random Forest is and how it...
4,Random Forest,Can you tell me what a Random Forest is and ho...
...,...,...
95,Random Forest,"Is it possible to discuss the number of trees,..."
96,Random Forest,"Is it possible to discuss the number of trees,..."
97,Random Forest,"Is it possible to discuss the number of trees,..."
98,Random Forest,"Is it possible to discuss the number of trees,..."


In [10]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW

# Read your dataset
df = pd.read_csv('augmented_questions.csv')

class InterviewQuestionsDataset(Dataset):
    def __init__(self, tokenizer, df, max_length=512):
        self.tokenizer = tokenizer
        self.data = df
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        input_text = f"topic: {row['topic']} </s>"
        target_text = row['Questions']

        source = self.tokenizer.encode_plus(input_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")
        target = self.tokenizer.encode_plus(target_text, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt")

        return source['input_ids'].squeeze(), target['input_ids'].squeeze()

# Prepare your data
tokenizer = T5Tokenizer.from_pretrained('t5-small')
dataset = InterviewQuestionsDataset(tokenizer, df)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Load T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-small')
model.train()

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids, labels = batch
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}: Loss {loss.item()}")

# Save the model
model.save_pretrained('./my_t5_model')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:303: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Epoch 1: Loss 0.2739100158214569
Epoch 2: Loss 0.18019282817840576
Epoch 3: Loss 0.22801637649536133


In [6]:
augmented_questions = pd.read_csv('augmented_questions.csv')

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load the fine-tuned model and tokenizer
model_path = 'my_t5_model'
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Prepare the input text
input_topic = "random forest"
input_text = f"topic: {input_topic} </s>"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate the question
outputs = model.generate(input_ids)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Generated Interview Question:", generated_text)